## Welcome to your notebook.


#### Run this cell to connect to your GIS and get started:

In [152]:
from arcgis.gis import GIS, features
gis = GIS("home")

#### Now you are ready to start!

In [73]:
import requests
import json
import numpy as np
import pandas as pd
from shapely.geometry import Point
import shapefile

In [4]:
# otrzymanie listy wszystkich stanowisk pomiarowych
response = requests.get("https://api.gios.gov.pl/pjp-api/rest/station/findAll")
responseJSON = json.loads(response.text)

In [5]:
#print(responseJSON)
#pobranie listy id stacji 
stationId = [station['id'] for i, station in enumerate(responseJSON)] #id stacji
stationLat = [station['gegrLat'] for i, station in enumerate(responseJSON)] # wspolzendne stacji, są one w kolejnosci id stacji więc ok
stationLon = [station['gegrLon'] for i, station in enumerate(responseJSON)]
print(stationId)

[114, 117, 129, 52, 109, 11, 16, 38, 70, 74, 84, 134, 9153, 11254, 11934, 12054, 12055, 17478, 156, 158, 16216, 16613, 295, 296, 314, 355, 10058, 291, 319, 322, 331, 562, 9913, 11358, 16271, 17160, 590, 10374, 568, 584, 600, 8976, 10018, 16413, 16913, 671, 10125, 17159, 17179, 631, 638, 646, 659, 665, 678, 684, 870, 882, 10005, 10030, 17182, 944, 16493, 952, 902, 920, 946, 950, 9218, 10834, 16495, 17438, 986, 987, 805, 809, 813, 814, 837, 841, 848, 834, 10554, 11855, 789, 10158, 798, 800, 842, 845, 853, 856, 9000, 11278, 11455, 11457, 11794, 16406, 11195, 16196, 16497, 769, 778, 785, 10794, 12016, 12138, 877, 861, 11554, 11616, 400, 401, 402, 10121, 10123, 10139, 10447, 11303, 444, 10120, 426, 9173, 9175, 10414, 10438, 609, 11174, 11814, 612, 618, 9994, 11154, 11916, 732, 736, 16242, 725, 740, 741, 742, 743, 750, 437, 443, 449, 455, 459, 9179, 10119, 10124, 10446, 10814, 11301, 11434, 16753, 16894, 17138, 530, 538, 550, 552, 10955, 10956, 497, 501, 515, 460, 466, 471, 485, 488, 517, 97

In [6]:
#pobranie danych dla całej polski
data = []
for i in range(0, len(stationId)):
    link=f"https://api.gios.gov.pl/pjp-api/rest/station/sensors/{stationId[i]}"
    response_s = requests.get(link)
    responseJSON_s = json.loads(response_s.text)
    data.append(responseJSON_s)
    
print(data) #informacje o czujnikach na stacjach (co stacje mierzą)

[[{'id': 642, 'stationId': 114, 'param': {'paramName': 'dwutlenek azotu', 'paramFormula': 'NO2', 'paramCode': 'NO2', 'idParam': 6}}, {'id': 644, 'stationId': 114, 'param': {'paramName': 'ozon', 'paramFormula': 'O3', 'paramCode': 'O3', 'idParam': 5}}], [{'id': 672, 'stationId': 117, 'param': {'paramName': 'dwutlenek siarki', 'paramFormula': 'SO2', 'paramCode': 'SO2', 'idParam': 1}}, {'id': 658, 'stationId': 117, 'param': {'paramName': 'benzen', 'paramFormula': 'C6H6', 'paramCode': 'C6H6', 'idParam': 10}}, {'id': 660, 'stationId': 117, 'param': {'paramName': 'tlenek węgla', 'paramFormula': 'CO', 'paramCode': 'CO', 'idParam': 8}}, {'id': 665, 'stationId': 117, 'param': {'paramName': 'dwutlenek azotu', 'paramFormula': 'NO2', 'paramCode': 'NO2', 'idParam': 6}}, {'id': 667, 'stationId': 117, 'param': {'paramName': 'ozon', 'paramFormula': 'O3', 'paramCode': 'O3', 'idParam': 5}}, {'id': 670, 'stationId': 117, 'param': {'paramName': 'pył zawieszony PM2.5', 'paramFormula': 'PM2.5', 'paramCode': 

In [7]:
#filtracja danych 
#lista stationId, Id (czujnika), paramCode czyli co czujnik miezy 
sensor_list = []

for sublist in data:
    for item in sublist:
        # Wyodrębnij potrzebne informacje
        station_id = item['stationId']
        sensor_id = item['id']
        param_code = item['param']['paramCode']

        # Dodaj do listy wynikowej
        sensor_list.append([station_id, sensor_id, param_code])

# Wyświetl uzyskaną listę
for row in sensor_list:
    print(row)

[114, 642, 'NO2']
[114, 644, 'O3']
[117, 672, 'SO2']
[117, 658, 'C6H6']
[117, 660, 'CO']
[117, 665, 'NO2']
[117, 667, 'O3']
[117, 670, 'PM2.5']
[117, 14395, 'PM10']
[129, 737, 'CO']
[129, 740, 'NO2']
[129, 744, 'PM2.5']
[52, 282, 'C6H6']
[52, 285, 'CO']
[52, 291, 'NO2']
[52, 293, 'O3']
[52, 297, 'SO2']
[52, 14397, 'PM10']
[109, 605, 'C6H6']
[109, 608, 'CO']
[109, 614, 'NO2']
[109, 616, 'O3']
[109, 618, 'PM10']
[109, 621, 'SO2']
[11, 50, 'NO2']
[11, 52, 'O3']
[11, 54, 'PM10']
[16, 101, 'PM10']
[38, 221, 'NO2']
[38, 223, 'O3']
[38, 224, 'PM10']
[38, 25988, 'PM2.5']
[70, 397, 'NO2']
[70, 400, 'PM10']
[70, 20214, 'O3']
[74, 441, 'NO2']
[74, 444, 'O3']
[74, 450, 'SO2']
[84, 493, 'O3']
[134, 14735, 'C6H6']
[9153, 14727, 'NO2']
[9153, 14729, 'O3']
[9153, 14730, 'PM10']
[9153, 14731, 'PM2.5']
[9153, 14733, 'SO2']
[9153, 14734, 'C6H6']
[9153, 14707, 'CO']
[11254, 20210, 'PM10']
[11934, 21368, 'NO2']
[11934, 21370, 'PM10']
[11934, 21371, 'O3']
[12054, 21635, 'PM10']
[12054, 21657, 'PM2.5']
[1205

In [8]:
#sortowanie wynikow
sensor_NO2 = []
sensor_PM10 = []
sensor_PM25 = []

sensor_NO2 = [row for row in sensor_list if row[2] == 'NO2']
sensor_PM10 = [row for row in sensor_list if row[2] == 'PM10']
sensor_PM25 = [row for row in sensor_list if row[2] == 'PM2.5']

print(sensor_NO2)

[[114, 642, 'NO2'], [117, 665, 'NO2'], [129, 740, 'NO2'], [52, 291, 'NO2'], [109, 614, 'NO2'], [11, 50, 'NO2'], [38, 221, 'NO2'], [70, 397, 'NO2'], [74, 441, 'NO2'], [9153, 14727, 'NO2'], [11934, 21368, 'NO2'], [12055, 29112, 'NO2'], [156, 959, 'NO2'], [158, 991, 'NO2'], [16216, 26088, 'NO2'], [295, 2064, 'NO2'], [296, 2083, 'NO2'], [314, 2195, 'NO2'], [355, 2373, 'NO2'], [10058, 16231, 'NO2'], [291, 2031, 'NO2'], [319, 2219, 'NO2'], [322, 2242, 'NO2'], [331, 16638, 'NO2'], [9913, 16037, 'NO2'], [590, 16160, 'NO2'], [568, 3839, 'NO2'], [584, 21351, 'NO2'], [16913, 27813, 'NO2'], [671, 4383, 'NO2'], [10125, 16319, 'NO2'], [638, 4192, 'NO2'], [659, 4278, 'NO2'], [665, 4333, 'NO2'], [882, 16424, 'NO2'], [10005, 16216, 'NO2'], [10030, 16181, 'NO2'], [17182, 28520, 'NO2'], [944, 6081, 'NO2'], [16493, 26990, 'NO2'], [952, 6161, 'NO2'], [902, 5899, 'NO2'], [920, 5979, 'NO2'], [946, 6104, 'NO2'], [950, 6140, 'NO2'], [10834, 17957, 'NO2'], [986, 6348, 'NO2'], [987, 6379, 'NO2'], [805, 5281, 'NO

In [9]:
#pobranie danych z czujnikow


data_NO2 = [] #wartosci z czujnikow
data_PM10 = []
data_PM25 = []

for i in range(0, len(sensor_NO2)):
    link=f"https://api.gios.gov.pl/pjp-api/rest/data/getData/{sensor_NO2[i][1]}"
    response_n = requests.get(link)
    responseJSON_n = json.loads(response_n.text)
    data_NO2.append([sensor_NO2[i][0], responseJSON_n])#dodaje tu jeszcze do wynikow id stacji, zeby latwo powiazac ze wspolrzednymi 
    
for i in range(0, len(sensor_PM10)):
    link=f"https://api.gios.gov.pl/pjp-api/rest/data/getData/{sensor_PM10[i][1]}"
    response_10 = requests.get(link)
    responseJSON_10 = json.loads(response_10.text)
    data_PM10.append([sensor_PM10[i][0], responseJSON_10])
    
for i in range(0, len(sensor_PM25)):
    link=f"https://api.gios.gov.pl/pjp-api/rest/data/getData/{sensor_PM25[i][1]}"
    response_25 = requests.get(link)
    responseJSON_25 = json.loads(response_25.text)
    data_PM25.append([sensor_PM25[i][0], responseJSON_25])
    
print(data_PM10)
    


[[117, {'key': 'PM10', 'values': [{'date': '2023-12-26 14:00:00', 'value': 4.07855}, {'date': '2023-12-26 13:00:00', 'value': 4.45654}, {'date': '2023-12-26 12:00:00', 'value': 3.72711}, {'date': '2023-12-26 11:00:00', 'value': 3.22124}, {'date': '2023-12-26 10:00:00', 'value': 3.29261}, {'date': '2023-12-26 09:00:00', 'value': 3.0942}, {'date': '2023-12-26 08:00:00', 'value': 2.86445}, {'date': '2023-12-26 07:00:00', 'value': 2.94863}, {'date': '2023-12-26 06:00:00', 'value': 3.09625}, {'date': '2023-12-26 05:00:00', 'value': 3.84082}, {'date': '2023-12-26 04:00:00', 'value': 5.04191}, {'date': '2023-12-26 03:00:00', 'value': 4.72338}, {'date': '2023-12-26 02:00:00', 'value': 4.68391}, {'date': '2023-12-26 01:00:00', 'value': 4.06282}, {'date': '2023-12-26 00:00:00', 'value': 3.75595}, {'date': '2023-12-25 23:00:00', 'value': 3.63982}, {'date': '2023-12-25 22:00:00', 'value': 3.73669}, {'date': '2023-12-25 21:00:00', 'value': 3.67368}, {'date': '2023-12-25 20:00:00', 'value': 3.82293}

In [161]:
for i in range(0, len(data_PM10)):
    if data_PM10[i][1]['values'] == []:
        del data_PM10[i]

In [162]:
data_PM10_df = pd.DataFrame({'ID_stacji' : [d[0] for d in data_PM10], 'Data_pom_pm25' : [data_PM10[i][1]['values'][1]['date'] for i in range(0, len(data_PM10))], 'Pom_pm25' : [data_PM10[i][1]['values'][1]['value'] for i in range(0, len(data_PM10))]}) 


In [51]:
stations = pd.DataFrame({'ID_stacji' : stationId, 'Lat_stacji' : stationLat, 'Lon_stacji' : stationLon})

In [62]:
stations = stations.join(data_PM10_df.set_index('ID_stacji'), on='ID_stacji', lsuffix='id')

In [70]:
stations['geometry'] = [Point(xy) for xy in zip(stations.Lon_stacji, stations.Lat_stacji)] 

In [164]:
stations

,ID_stacji,Lat_stacji,Lon_stacji,Data_pom_pm25,Pom_pm25,geometry
0,114,51.115933,17.141125,NaN,NaN,POINT (17.141125 51.115933)
1,117,51.129378,17.029250,2023-12-26 13:00:00,4.45654,POINT (17.02925 51.129378)
2,129,51.086225,17.012689,NaN,NaN,POINT (17.012689 51.086225)
3,52,51.204503,16.180513,2023-12-26 13:00:00,6.50414,POINT (16.180513 51.204503)
4,109,50.768729,16.269677,2023-12-26 13:00:00,10.84360,POINT (16.269677 50.768729)
...,...,...,...,...,...,...
198,966,54.193986,16.172544,2023-12-26 13:00:00,7.24050,POINT (16.172544 54.193986)
199,11336,54.194114,16.211672,NaN,NaN,POINT (16.211672 54.194114)
200,961,53.122325,14.382245,NaN,NaN,POINT (14.382245 53.122325)
201,983,53.698902,16.704556,2023-12-26 13:00:00,7.21556,POINT (16.704556 53.698902)


In [91]:
pd.reset_option('all')

In [120]:
output_shapefile_path = '/arcgis/home/shapefiles_PM_25'
shp_writer = shapefile.Writer(output_shapefile_path)

In [127]:
# Define the fields
shp_writer.field('ID_stacji', 'N')
shp_writer.field('Data_pom_pm25', 'S')
shp_writer.field('Pom_pm25', 'N')

# Set the bounding box explicitly
shp_writer.bbox = [stations['Lon_stacji'].min(), stations['Lat_stacji'].min(), stations['Lon_stacji'].max(), stations['Lat_stacji'].max()]


In [122]:
# Add geometry to the shapefile
for i, row in stations.iterrows():
    lat = float(row['Lat_stacji'])
    lon = float(row['Lon_stacji'])
    
    pom_pm25_value = row['Pom_pm25'] if not pd.isna(row['Pom_pm25']) else None
    data_pom_pm25_value = row['Data_pom_pm25'] if not pd.isna(row['Pom_pm25']) else None
    
    # Add attributes and geometry to the shapefile
    shp_writer.record(row['ID_stacji'], data_pom_pm25_value, pom_pm25_value)
    shp_writer.point(lon, lat)

In [160]:
# Save the shapefile
shp_writer.close()

TypeError: 'list' object is not callable